k-vizinhos mais próximos e hiperparâmetros
==========================================



## O algoritmo dos $k$-vizinhos mais próximos ($k$​-NN)



O algoritmo $k$​-NN é um algoritmo de aprendizado de máquina relativamente simples baseado na hipótese que dados similares ocupam regiões próximas no espaço de entrada.

Quando um modelo treinado por este algoritmo é usado para realizar a previsão de um certo exemplo $x$, ele checa a distância deste exemplo $x$ com todos os exemplos usados para treinar o modelo. Os $k$ exemplos mais próximos de $x$ são usados para calcular a previsão do exemplo $x$.

Para uma fundamentação teórica mais aprofundada, leia as páginas 53 a 58 do livro [FACELI, Katti; LORENA, AC; GAMA, João; et al. Inteligência Artificial: uma abordagem de Aprendizado de Máquina. 2. ed., 2021]. Vale a pena também assistir ao vídeo do [StatQuest](https://www.youtube.com/watch?v=HVXime0nQeI) e ao vídeo do [Prof. José Segundo](https://www.youtube.com/watch?v=4Dpf76AB9Js).

Para usar o `scikit-learn` para treinar um modelo $k$​-NN nós vamos seguir o mesmo procedimento que usamos no notebook sobre modelo linear (se não leu ele, é recomendado parar aqui e retortar após a leitura).

Vamos primeiro carregar os dados.



In [1]:
import seaborn as sns
from sklearn.model_selection import train_test_split

TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

# observe que usamos o .values aqui pois queremos apenas os valores
X_treino = df_treino.reindex(FEATURES, axis=1).values
y_treino = df_treino.reindex(TARGET, axis=1).values
X_teste = df_teste.reindex(FEATURES, axis=1).values
y_teste = df_teste.reindex(TARGET, axis=1).values

Agora vamos treinar o modelo!



In [2]:
from sklearn.neighbors import KNeighborsRegressor

# cria o modelo
modelo_knn = KNeighborsRegressor()

# treina o modelo
modelo_knn.fit(X_treino, y_treino)

# realiza uma previsão usando o modelo treinado
previsao = modelo_knn.predict(X_teste)
print(previsao)

[[ 943. ]
 [ 641.2]
 [ 768. ]
 ...
 [ 508.6]
 [ 509.8]
 [1041.4]]


Vamos checar a performance do nosso modelo $k$-NN.



In [3]:
from sklearn.metrics import mean_squared_error

y_verdadeiro = y_teste
y_previsao = modelo_knn.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo k-NN foi de {RMSE} dólares.")

O RMSE do modelo k-NN foi de 1471.3357190226614 dólares.


O RMSE do modelo $k$​-NN foi de 1471 dólares, o que é levemente melhor do que o RMSE do modelo linear que foi de 1477 dólares. Provavelmente estes valores são estatisticamente iguais dentro da incerteza, então não vamos comemorar ainda.



## Hiperparâmetros



Quem sabe você esteja se perguntando &ldquo;ué&#x2026; mas se o modelo $k$​-NN usa os $k$ vizinhos mais próximos para fazer sua previsão, onde é que isso foi configurado no exemplo acima?&rdquo;.

Na verdade nós usamos um valor de $k$ igual a 5 no exemplo acima, pois esse é o valor padrão deste modelo no `scikit-learn`. No entanto, nada nos impede de usar outros valores! Vamos ver o que acontece quando usamos um valor de $k$ igual a 1, igual a 10 e igual a 100.



In [4]:
NUM_VIZINHOS = [1, 10, 100]

for n in NUM_VIZINHOS:
    modelo_knn = KNeighborsRegressor(n_neighbors=n)

    modelo_knn.fit(X_treino, y_treino)

    y_verdadeiro = y_teste
    y_previsao = modelo_knn.predict(X_teste)
    RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)
    print(f"O RMSE do modelo k-NN usando k={n} foi de {RMSE} dólares.")

O RMSE do modelo k-NN usando k=1 foi de 1767.1024604343038 dólares.
O RMSE do modelo k-NN usando k=10 foi de 1415.3859761300655 dólares.
O RMSE do modelo k-NN usando k=100 foi de 1389.2856173027792 dólares.


Parece que conseguimos um modelo significantemente melhor que o linear! O modelo $k$​-NN usando 100 vizinhos teve um RMSE de 1389 dólares!

A lição importante aqui é que existem certos parâmetros que usamos para controlar como nossos algoritmos de aprendizado de máquina vão induzir um modelo durante o seu treino. Neste caso, o valor de $k$ é um desses parâmetros de controle. Veja que o valor de $k$, uma vez definido, não se altera durante o treinamento do modelo. Chamamos de <u>hiperparâmetros</u> o conjunto de parâmetros que definimos antes do treino dos modelos e que não se alteram durante o treino.

O algoritmo $k$​-NN tem seis hiperparâmetros que podem ser controlados pelo usuário, para checá-los basta dar um pulinho na nossa velha amiga, a [documentação](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html).

Para cada conjunto de {dados, algoritmo de machine learning} existe um conjunto de hiperparâmetros que melhor modela o problema (entregando a melhor performance). Infelizmente, não existe uma forma determinística além a estratégia de força bruta para encontrar este conjunto ótimo de hiperparâmetros&#x2026; por sorte, muitas vezes um conjunto razoavelmente bom de hiperparâmetros pode ser obtido usando estratégias de otimização. Damos o nome de <u>otimização de hiperparâmetros</u> (*hiperparameter tuning*) ao ato de se buscar um bom conjunto de hiperparâmetros para seu problema.

